In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Vector Search - RAG Demo



### Prerequisites

This tutorial requires a Google Cloud project that is linked with a billing account. To create a new project, take a look at [this document](https://cloud.google.com/vertex-ai/docs/start/cloud-environment) to create a project and setup a billing account for it.

### Choose the runtime environment

This tutorial can be run on either Google Colab or [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).

- To use Colab: Click [this link](https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/embeddings/vector-search-quickstart.ipynb) to open the tutorial in Colab.

- To use Workbench: If it is the first time to use Workbench in your Google Cloud project, open [the Workbench console](https://console.cloud.google.com/vertex-ai/workbench) and click ENABLE button to enable Notebooks API. The click [this link](https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/embeddings/vector-search-quickstart.ipynb),  and select an existing notebook or create a new notebook.


### How much will this cost?

To go through this tutorial, it will cost roughly a few US dollars. The pricing of the Cloud services we will use in this tutorial are available in the following pages:

- [Vertex AI Embeddings for Text](https://cloud.google.com/vertex-ai/pricing#generative_ai_models)
- [Vertex AI Vector Search](https://cloud.google.com/vertex-ai/pricing#matchingengine)
- [Vertex AI Workbench](https://cloud.google.com/vertex-ai/pricing#notebooks) if you use one

You can use the [Pricing Calculator](https://cloud.google.com/products/calculator) to generate a cost estimate based on your projected usage.

### **Warning: delete your objects after the tutorial**

In case you are using your own Cloud project, please make sure to delete all the Indexes and Index Endpoints (and the Workbench instance if you use one) after finishing this tutorial. Otherwise the remaining assets would incur unexpected costs.


## Setup

Before get started with the Vertex AI services, we need to setup the following.

* Install Python SDK
* Environment variables
* Authentication (Colab only)
* Enable APIs
* Set IAM permissions

### Install the Vertex AI SDK

Vertex AI APIs can be accessed with multiple ways including REST API and Python SDK. In this tutorial we will use the SDK.

In [ ]:
%pip install --upgrade --quiet --user google-cloud-aiplatform

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

For Colab users: you will see "Your session crushed" message when restarting the kernel, but it is expected. Continue to the following cells.

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. In Colab or Colab Enterprise, you might see an error message that says "Your session crashed for an unknown reason." This is expected. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Environment variables

Sets environment variables. If asked, replace the following `[your-project-id]` with your project ID and run it.

In [ ]:
# get project ID
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"
if PROJECT_ID == "(unset)":
    print(f"Please set the project ID manually below")

In [ ]:
# define project information
if PROJECT_ID == "(unset)":
    PROJECT_ID = "uk-con-gcp-sbx-ukat01-012822"  # @param {type:"string"}

# generate an unique id for this session
from datetime import datetime

UID = datetime.now().strftime("%m%d%H%M")

### Authentication (Colab only)

If you are running this notebook on Colab, you will need to run the following cell authentication. This step is not required if you are using Vertex AI Workbench as it is pre-authenticated.

In [ ]:
import sys

# if it's Colab runtime, authenticate the user with Google Cloud
if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set IAM permissions

Also, we need to add access permissions to the default service account for using those services.

- Go to [the IAM page](https://console.cloud.google.com/iam-admin/) in the Console
- Look for the principal for default compute service account. It should look like: `<project-number>-compute@developer.gserviceaccount.com`
- Click the edit button at right and click `ADD ANOTHER ROLE` to add `Vertex AI User` and `Service Usage Admin` roles to the account.


### Enable APIs

Run the following to enable APIs for Compute Engine and Vertex AI with this Google Cloud project.

In [ ]:
! gcloud services enable compute.googleapis.com aiplatform.googleapis.com --project "{PROJECT_ID}"

In [ ]:
!pip install PyPDF2

In [ ]:
from google.cloud import storage
from vertexai.language_models import TextEmbeddingModel
import PyPDF2
import re
import os
import random
import json
import uuid

pdf_path="banking-demo.pdf"
bucket_name = "bank-product-rec"
embed_file_path = "bank_embeddings.json"
sentence_file_path = "bank_sentences.json"

! gsutil cp "gs://bank-product-rec/banking-demo.pdf" .

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
def extract_sentences_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            if page.extract_text() is not None:
                text += page.extract_text() + " "
    sentences = [sentence.strip() for sentence in text.split('. ') if sentence.strip()]
    return sentences

In [ ]:
def generate_text_embeddings(sentences) -> list:
  model = TextEmbeddingModel.from_pretrained("textembedding-gecko@003")
  embeddings = model.get_embeddings(sentences)
  vectors = [embedding.values for embedding in embeddings]
  return vectors

In [ ]:
def generate_and_save_embeddings(pdf_path, sentence_file_path, embed_file_path):
    def clean_text(text):
        cleaned_text = re.sub(r'\u2022', '', text)  # Remove bullet points
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # Remove extra whitespaces and strip
        return cleaned_text

    sentences = extract_sentences_from_pdf(pdf_path)
    if sentences:
        embeddings = generate_text_embeddings(sentences)

        with open(embed_file_path, 'w') as embed_file, open(sentence_file_path, 'w') as sentence_file:
            for sentence, embedding in zip(sentences, embeddings):
                cleaned_sentence = clean_text(sentence)
                id = str(uuid.uuid4())

                embed_item = {"id": id, "embedding": embedding}
                sentence_item = {"id": id, "sentence": cleaned_sentence}

                json.dump(sentence_item, sentence_file)
                sentence_file.write('\n')
                json.dump(embed_item, embed_file)
                embed_file.write('\n')

In [ ]:
generate_and_save_embeddings(pdf_path,sentence_file_path,embed_file_path)

## Build and Deploy a Vector Search Index

### Create Index

Now it's ready to load the embeddings to Vector Search. Its APIs are available under the [aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform) package of the SDK.

Create an [MatchingEngineIndex](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.MatchingEngineIndex) with its `create_tree_ah_index` function (Matching Engine is the previous name of Vector Search).

In [ ]:
# create Index
INDEX_NAME = f"vs-bank-prod-index-{UID}"

my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=INDEX_NAME,
    dimensions=768,
    approximate_neighbors_count=10,
    index_update_method="STREAM_UPDATE",
)

By calling the `create_tree_ah_index` function, it starts building an Index.  This will take a few minutes if the dataset is small, otherwise about 60 minutes or more depending on the size of the dataset. You can check status of the index creation on [the Vector Search Console > INDEXES tab](https://console.cloud.google.com/vertex-ai/matching-engine/indexes).

#### The parameters for creating index

- `dimensions`: Dimension size of each embedding. In this case, it is 768 as we are using the embeddings from the Text Embeddings API.
- `approximate_neighbors_count`: how many similar items we want to retrieve in typical cases
- `index_update_method`: `STREAM_UPDATE` allows real time update to the index.

See [the document](https://cloud.google.com/vertex-ai/docs/vector-search/create-manage-index) for more details on creating Index and the parameters.

### Create Index Endpoint and deploy the Index

To use the Index, you need to create an [Index Endpoint](https://cloud.google.com/vertex-ai/docs/vector-search/deploy-index-public). It works as a server instance accepting query requests for your Index.

In [ ]:
# create IndexEndpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"vs-bank-prod-index-endpoint-{UID}", public_endpoint_enabled=True
)

This tutorial utilizes a [Public Endpoint](https://cloud.google.com/vertex-ai/docs/vector-search/setup/setup#choose-endpoint) and does not support [Virtual Private Cloud (VPC)](https://cloud.google.com/vpc/docs/private-services-access). Unless you have a specific requirement for VPC, we recommend using a Public Endpoint. Despite the term "public" in its name, it does not imply open access to the public internet. Rather, it functions like other endpoints in Vertex AI services, which are secured by default through IAM. Without explicit IAM permissions, as we have previously established, no one can access the endpoint.

With the Index Endpoint, deploy the Index by specifying an unique deployed index ID.

In [ ]:
DEPLOYED_INDEX_ID = f"vs_bank_prod_deployed_{UID}"

**Note:** There is a difference between INDEX_NAME and DEPLOYED_INDEX_ID. The first one may include dashes (-), while the second one only allows underscores (_).

In [ ]:
# deploy the Index to the Index Endpoint
my_index_endpoint.deploy_index(index=my_index, deployed_index_id=DEPLOYED_INDEX_ID)

If it is the first time to deploy an Index to an Index Endpoint, it will take around 30 minutes to automatically build and initiate the backend for it. After the first deployment, it will finish in seconds. To see the status of the index deployment, open [the Vector Search Console > INDEX ENDPOINTS tab](https://console.cloud.google.com/vertex-ai/matching-engine/index-endpoints) and click the Index Endpoint.

## Streaming update to the vector search index

At this time, the vector search index still empty. The following code will insert the product embeddings from the JSONL file by using `upsert_datapoints` method.

In [ ]:
import json

# read product-embs.json and put them to a list
with open(embed_file_path) as f:
    datapoints = []
    for line in f:
        item = json.loads(line)
        datapoints.append(
            {"datapoint_id": str(item["id"]), "feature_vector": item["embedding"]}
        )

In [ ]:
# insert datapoints to the index
for i in range(0, len(datapoints), 1000):
    my_index.upsert_datapoints(datapoints=datapoints[i : i + 1000])

The `upsert_datapoints()` method takes a list of dicts where each dict should have two properties: `datapoint_id` for ID of the item, and `feature_vector` for embeddings of the item. The method can take up to 1000 items at a time.

The `upsert_datapoints()` method adds the items in the list to the vector search index in near real-time. As long as you are adding a few items at a time, it should be reflected to the query results in a couple of seconds. So this is suitable for use cases such as product catalogs where the sellers want to add or update many items every day and want the search system to reflect it instantly.

See [the document](https://cloud.google.com/vertex-ai/docs/vector-search/update-rebuild-index#upsert-streaming) for more details on streaming update.

## Run a Query with Vector Search

Finally it's ready to use Vector Search. To run a query, you need to get an embedding for a query item. For example, if you like to find products with similar names to "cloudveil women's excursion short", you need to get the embedding for it before running a query. In this tutorial, we will get the embedding from the `product-embs.json` file as follows:

In [ ]:
# # read product-embs.json and build dicts of product names and embeddings
# with open(embed_file_path) as f:
#     product_names = {}
#     product_embs = {}
#     for line in f:
#         item = json.loads(line)
#         product_names[str(item["id"])] = item["name"]
#         product_embs[str(item["id"])] = item["embedding"]

In [ ]:
def generate_text_embeddings(sentences) -> list:
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko@003")
    embeddings = model.get_embeddings(sentences)
    vectors = [embedding.values for embedding in embeddings]
    return vectors

With the `product_embs` dict, you can specify a product ID to get an embedding for it.

In [ ]:
# get the embedding for ID 6523 "cloudveil women's excursion short"
# you can also try with other IDs such as 12711, 18090, 19536 and 11863
# query_emb = product_embs["6523"]
#query=["How many days of unpaid leave in an year"]
#query=["Allowed cost of online course"]
#query=["process for applying sick leave"]
query=["savings account with the best interest rate and suitable for someone under 16"]
qry_emb=generate_text_embeddings(query)

In [ ]:
print(qry_emb[0])

### Run a Query

Then, pass the embedding to `find_neighbors` function to find similar product names.

In [ ]:
from vertexai.preview.generative_models import GenerativeModel, Part

my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(index_endpoint_name="4381980447198937088")

# run query
response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID, queries=[qry_emb[0]], num_neighbors=10
)

# show the results
# for idx, neighbor in enumerate(response[0]):
#     print(f"{neighbor.distance:.2f} {product_names[neighbor.id]}")
matching_ids = [neighbor.id for sublist in response for neighbor in sublist]

In [ ]:
data = []
with open(sentence_file_path) as file:
  for line in file:
    d = json.loads(line)
    data.append(d)

In [ ]:
def generate_context(ids,data):
  concatenated_names = ''
  for id in ids:
    for entry in data:
      # print(entry)
      if entry['id'] == id:
        concatenated_names += entry['sentence'] + "\n"
  return concatenated_names.strip()

context = generate_context(matching_ids,data)
print(context)

In [ ]:
model = GenerativeModel("gemini-pro")
prompt=f"Based on the context delimited in backticks, answer the query. ```{context}``` {query}"

In [ ]:
print(prompt)

In [ ]:
chat = model.start_chat(history=[])
response = chat.send_message(prompt)
print(response.text)

The `find_neighbors` function only takes milliseconds to fetch the similar items even when you have billions of items on the Index, thanks to the ScaNN algorithm. Vector Search also supports [autoscaling](https://cloud.google.com/vertex-ai/docs/vector-search/deploy-index-public#autoscaling) which can automatically resize the number of nodes based on the demands of your workloads.

# IMPORTANT: Cleaning Up

In case you are using your own Cloud project, not a temporary project on Qwiklab, please make sure to delete all the Indexes and Index Endpoints after finishing this tutorial. Otherwise the remaining objects would **incur unexpected costs**.

If you used Workbench, you may also need to delete the Notebooks from [the console](https://console.cloud.google.com/vertex-ai/workbench).

In [ ]:
# wait for a confirmation
input("Press Enter to delete Index Endpoint and Index:")

# delete Index Endpoint
my_index_endpoint.undeploy_all()
my_index_endpoint.delete(force=True)

# delete Index
my_index.delete()

## Utilities

It can take some time to create or deploy indexes, and in that time you might lose connection with the Colab runtime. If you lose connection, instead of creating or deploying your new index again, you can check [the Vector Search Console](https://console.cloud.google.com/vertex-ai/matching-engine/index-endpoints) and use the existing ones to continue.


### Get an existing Index

To get an index object that already exists, replace the following `[numeric-index-id]` with the index ID and run the cell. You can check the ID on [the Vector Search Console > INDEXES tab](https://console.cloud.google.com/vertex-ai/matching-engine/indexes).

In [ ]:
my_index_id = "5689906703043133440"  # @param {type:"string"}
my_index = aiplatform.MatchingEngineIndex(my_index_id)

### Get an existing Index Endpoint

To get an index endpoint object that already exists, replace the following `[numeric-index-endpoint-id]` with the index endpoint ID and run the cell. You can check the ID on [the Vector Search Console > INDEX ENDPOINTS tab](https://console.cloud.google.com/vertex-ai/matching-engine/index-endpoints).

In [ ]:
my_index_endpoint_id = "7802517140744962048"  # @param {type:"string"}
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(my_index_endpoint_id)